In [1]:
%load_ext autoreload
# %reload_ext autoreload
%autoreload 2


In [2]:
from methylVA.mnist.dataset import get_methyl_data_loaders

data_id = 0.05
batch_size = 128

train_data_path = f"../data/dimension_reduction/highly_variable_features/train_data_filtered_{data_id}.pkl"
train_metadata_path = f"../data/dimension_reduction/highly_variable_features/train_metadata_with_labels.pkl"
test_data_path = f"../data/dimension_reduction/highly_variable_features/test_data_filtered_{data_id}.pkl"
test_metadata_path = f"../data/dimension_reduction/highly_variable_features/test_metadata_with_labels.pkl"


train_loader, test_loader = get_methyl_data_loaders(
    train_data_path,
    train_metadata_path,
    test_data_path,
    test_metadata_path,
    batch_size=batch_size
)


Found NaN values in the data after conversion.
Found NaN values in the data after conversion.


In [3]:
data_batch, _ = next(iter(train_loader))


num_train_rows = len(train_loader.dataset)
num_test_rows = len(test_loader.dataset)

print("Number of features in each dataset:", data_batch.shape[1])
print("Number of rows in the training dataset:", num_train_rows)
print("Number of rows in the test dataset:", num_test_rows)

Number of features in each dataset: 30579
Number of rows in the training dataset: 33360
Number of rows in the test dataset: 3707


In [4]:
from datetime import datetime

import torch
from torch.utils.tensorboard import SummaryWriter

from methylVA.mnist.model import VAE
from methylVA.mnist.training import train, test

input_dim = data_batch.shape[1]
learning_rate = 1e-3
weight_decay = 1e-2
num_epochs = 100
latent_dim = 32
hidden_dim = 2048
kl_weight = 1.0
name = f'VAE_methyl_data_{data_id}_latent_{latent_dim}_kl_{kl_weight}'



In [5]:

writer_train = SummaryWriter(f'../experiments/{name}/train/{datetime.now().strftime("%Y%m%d-%H%M%S")}')
writer_test = SummaryWriter(f'../experiments/{name}/test/{datetime.now().strftime("%Y%m%d-%H%M%S")}')

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = VAE(input_dim=input_dim, latent_dim=latent_dim, hidden_dim=hidden_dim).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

In [7]:
model

VAE(
  (encoder): Sequential(
    (0): Linear(in_features=30579, out_features=2048, bias=True)
    (1): SiLU()
    (2): Linear(in_features=2048, out_features=1024, bias=True)
    (3): SiLU()
    (4): Linear(in_features=1024, out_features=512, bias=True)
    (5): SiLU()
    (6): Linear(in_features=512, out_features=256, bias=True)
    (7): SiLU()
    (8): Linear(in_features=256, out_features=64, bias=True)
  )
  (softplus): Softplus(beta=1.0, threshold=20.0)
  (decoder): Sequential(
    (0): Linear(in_features=32, out_features=256, bias=True)
    (1): SiLU()
    (2): Linear(in_features=256, out_features=512, bias=True)
    (3): SiLU()
    (4): Linear(in_features=512, out_features=1024, bias=True)
    (5): SiLU()
    (6): Linear(in_features=1024, out_features=2048, bias=True)
    (7): SiLU()
    (8): Linear(in_features=2048, out_features=30579, bias=True)
    (9): Sigmoid()
  )
)

In [ ]:
from methylVA.mnist.training import train, test


prev_updates = 0
for epoch in range(num_epochs):
    print(f'Epoch {epoch + 1}/{num_epochs}')
    prev_updates = train(model, train_loader, optimizer, prev_updates, writer=writer_train)
    test(model, test_loader, prev_updates, writer=writer_test)

Epoch 1/100


  0%|          | 1/261 [00:00<01:56,  2.24it/s]

Step 0, (N samples: 0), Loss: 21200.0156, (Recon: 21196.5586, KLD: 3.4579), Gradient norm: 37.3112


 39%|███▉      | 102/261 [00:16<00:24,  6.47it/s]

Step 100, (N samples: 12,800), Loss: 16289.8965, (Recon: 16254.0186, KLD: 35.8778), Gradient norm: 2319.5700


...

Testing: 100%|██████████| 29/29 [00:04<00:00,  6.39it/s]


====> Test set loss: 14255.8241, (BCE: 14232.3304, KLD: 23.4938)
Epoch 94/100


 11%|█         | 28/261 [00:07<01:02,  3.72it/s]

Step 24,300, (N samples: 3,110,400), Loss: 14049.5293, (Recon: 14025.6250, KLD: 23.9046), Gradient norm: 1284.9076


 31%|███       | 81/261 [00:21<00:47,  3.79it/s]

In [11]:
def pearson_correlation(original_x, x_hat, mask=None):
    
    # Calculate mean and standard deviation
    mean_x = torch.mean(original_x)
    mean_x_hat = torch.mean(x_hat)
    
    std_x = torch.std(original_x)
    std_x_hat = torch.std(x_hat)
    
    # Calculate covariance
    covariance = torch.mean((original_x - mean_x) * (x_hat - mean_x_hat))
    
    # Calculate Pearson correlation
    correlation = covariance / (std_x * std_x_hat)
    
    return correlation.item()

In [14]:
import torch

# Assuming `model` is your trained model and `dataloader` is your DataLoader
model.eval()  # Set model to evaluation mode

correlations = []

# Disable gradient calculation for efficiency
with torch.no_grad():
    for batch in test_loader:
        # Get the input data (original_x)
        original_x, _ = batch  # Adjust this if batch contains labels or other info
        original_x = original_x.to(device)  # Move to device if using GPU

        # Get the model output (x_hat)
        out = model(original_x)
        x_hat = out.x_recon
        
        # Compute Pearson correlation for the batch
        correlation = pearson_correlation(original_x, x_hat)
        correlations.append(correlation)
        
        # Print or store the correlation
        print("Pearson Correlation:", correlation)


Pearson Correlation: 0.957661509513855
Pearson Correlation: 0.950019896030426
Pearson Correlation: 0.9605104327201843
Pearson Correlation: 0.9542161226272583
Pearson Correlation: 0.951418936252594
Pearson Correlation: 0.9480416178703308
Pearson Correlation: 0.9556251168251038
Pearson Correlation: 0.9498641490936279
Pearson Correlation: 0.9456736445426941
Pearson Correlation: 0.9548927545547485
Pearson Correlation: 0.9457770586013794
Pearson Correlation: 0.9527332186698914
Pearson Correlation: 0.9417027831077576
Pearson Correlation: 0.9517192840576172
Pearson Correlation: 0.9572741985321045
Pearson Correlation: 0.9492014646530151
Pearson Correlation: 0.945146918296814
Pearson Correlation: 0.9474121332168579
Pearson Correlation: 0.9577855467796326
Pearson Correlation: 0.9552962779998779
Pearson Correlation: 0.9518298506736755
Pearson Correlation: 0.9525393843650818
Pearson Correlation: 0.9414599537849426
Pearson Correlation: 0.951117753982544
Pearson Correlation: 0.948650598526001
Pearso

In [17]:
correlations = []

# Disable gradient calculation for efficiency
with torch.no_grad():
    for batch in train_loader:
        # Unpack the batch to get the input data; adjust if there are labels
        original_x, _ = batch  # Adjust this if your batch contains more items
        original_x = original_x.to(device)  # Move input to device if using GPU

        # Get the model output for the entire batch
        out = model(original_x)
        x_hat_batch = out.x_recon

        # Compute Pearson correlation for each element in the batch
        for i in range(original_x.size(0)):  # Loop over each sample in the batch
            original_sample = original_x[i]  # Select the i-th sample
            x_hat_sample = x_hat_batch[i]    # Select the corresponding output sample

            # Compute Pearson correlation for the sample
            correlation = pearson_correlation(original_sample, x_hat_sample)
            correlations.append(correlation)
            # print(f"Sample {i} Pearson Correlation:", correlation)

# Optionally, compute average correlation across all elements
average_correlation = sum(correlations) / len(correlations)
print("Average Pearson Correlation across all samples:", average_correlation)


Average Pearson Correlation across all samples: 0.9529571182722096
